In [1]:
import pandas as pd
import numpy as np
import flask
import pymongo
import json
from datetime import datetime

In [2]:
#Name of the CSV file
state_data = 'NYT-data/nyt_state_data.csv'
county_data = 'NYT-data/nyt_county_data.csv'
lat_long_counties = 'NYT-data/counties_for_latlong.csv'

In [3]:
state_df = pd.read_csv(state_data)
county_df = pd.read_csv(county_data)

In [4]:
state_df = state_df.sort_values('date', ascending = False)
county_df = county_df.sort_values('date', ascending = False)

In [5]:
state_df['date'] = pd.to_datetime(state_df['date'], format= '%m/%d/%y')
state_df = state_df.sort_values('date', ascending = False)
county_df['date'] = pd.to_datetime(county_df['date'], format= '%m/%d/%y')
county_df = county_df.sort_values('date', ascending = False)

In [6]:
state_df['date']=state_df['date'].dt.strftime('%m/%d/%Y')
county_df['date']=county_df['date'].dt.strftime('%m/%d/%Y')

In [7]:
json_states = state_df.to_json(orient = 'records')
parsed = json.loads(json_states)
print(json.dumps(parsed, indent = 4, sort_keys= True))

[
    {
        "cases": 53732,
        "date": "07/30/2020",
        "deaths": 1634,
        "fips": 27,
        "state": "Minnesota"
    },
    {
        "cases": 461371,
        "date": "07/30/2020",
        "deaths": 6585,
        "fips": 12,
        "state": "Florida"
    },
    {
        "cases": 1966,
        "date": "07/30/2020",
        "deaths": 25,
        "fips": 15,
        "state": "Hawaii"
    },
    {
        "cases": 1323,
        "date": "07/30/2020",
        "deaths": 6,
        "fips": 66,
        "state": "Guam"
    },
    {
        "cases": 85762,
        "date": "07/30/2020",
        "deaths": 1565,
        "fips": 1,
        "state": "Alabama"
    },
    {
        "cases": 3557,
        "date": "07/30/2020",
        "deaths": 21,
        "fips": 2,
        "state": "Alaska"
    },
    {
        "cases": 170905,
        "date": "07/30/2020",
        "deaths": 3628,
        "fips": 4,
        "state": "Arizona"
    },
    {
        "cases": 41759,
        "date": 

In [8]:
json_counties = county_df.to_json(orient = 'records')
parsed = json.loads(json_counties)
#print(json.dumps(parsed, indent = 4, sort_keys= True))

In [9]:
from config import geocode_key

## Adding Lat/Long to json

In [10]:
state_df.head()

,date,state,fips,cases,deaths
8233,07/30/2020,Minnesota,27,53732,1634
8218,07/30/2020,Florida,12,461371,6585
8221,07/30/2020,Hawaii,15,1966,25
8220,07/30/2020,Guam,66,1323,6
8209,07/30/2020,Alabama,1,85762,1565


In [11]:
state_series=state_df['state']
state_series

8233     Minnesota
8218       Florida
8221        Hawaii
8220          Guam
8209       Alabama
           ...    
4       Washington
3         Illinois
2       Washington
1       Washington
0       Washington
Name: state, Length: 8264, dtype: object

In [12]:
state_list = state_series.unique().tolist()
state_list

['Minnesota',
 'Florida',
 'Hawaii',
 'Guam',
 'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Georgia',
 'Wyoming',
 'Wisconsin',
 'Mississippi',
 'Idaho',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Missouri',
 'Nebraska',
 'West Virginia',
 'Washington',
 'Pennsylvania',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Northern Mariana Islands',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Puerto Rico',
 'Montana',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virgin Islands',
 'Virginia',
 'Michigan',
 'Illinois']

In [13]:
state_coords_df=pd.DataFrame(state_list,columns=['State'])
state_coords_df

,State
0,Minnesota
1,Florida
2,Hawaii
3,Guam
4,Alabama
5,Alaska
6,Arizona
7,Arkansas
8,California
9,Colorado


In [14]:
name=[]
lat=[]
long=[]

In [15]:
import requests

In [16]:
for i in state_list:
    target_url=(f"https://maps.googleapis.com/maps/api/geocode/json?address={i}&key={geocode_key}")
    geodata=requests.get(target_url).json()
    state_name=geodata['results'][0]['address_components'][0]['long_name']
    name.append(state_name)
    
    latitude=geodata['results'][0]['geometry']['location']['lat']
    lat.append(latitude)
    
    longitude=geodata['results'][0]['geometry']['location']['lng']
    long.append( longitude)

In [17]:
print(len(name))
print(len(lat))
print(len(long))

55
55
55


In [18]:
state_coords_df['Latitude']=pd.DataFrame(lat)
state_coords_df['Longitude']=pd.DataFrame(long)
state_coords_df.head(5)

,State,Latitude,Longitude
0,Minnesota,46.729553,-94.685900
1,Florida,27.664827,-81.515754
2,Hawaii,19.896766,-155.582782
3,Guam,13.444304,144.793731
4,Alabama,32.318231,-86.902298


In [19]:
state_coords_df


,State,Latitude,Longitude
0,Minnesota,46.729553,-94.685900
1,Florida,27.664827,-81.515754
2,Hawaii,19.896766,-155.582782
3,Guam,13.444304,144.793731
4,Alabama,32.318231,-86.902298
5,Alaska,64.200841,-149.493673
6,Arizona,34.048928,-111.093731
7,Arkansas,35.201050,-91.831833
8,California,36.778261,-119.417932
9,Colorado,39.550051,-105.782067


In [20]:
state_df

,date,state,fips,cases,deaths
8233,07/30/2020,Minnesota,27,53732,1634
8218,07/30/2020,Florida,12,461371,6585
8221,07/30/2020,Hawaii,15,1966,25
8220,07/30/2020,Guam,66,1323,6
8209,07/30/2020,Alabama,1,85762,1565
...,...,...,...,...,...
4,01/24/2020,Washington,53,1,0
3,01/24/2020,Illinois,17,1,0
2,01/23/2020,Washington,53,1,0
1,01/22/2020,Washington,53,1,0


In [21]:
state_df = state_df.rename(columns = {'state':'State'})
state_final_df=pd.merge(state_df,state_coords_df, on='State')
state_final_df = state_final_df.drop_duplicates()

In [22]:
state_final_df

,date,State,fips,cases,deaths,Latitude,Longitude
0,07/30/2020,Minnesota,27,53732,1634,46.729553,-94.685900
1,07/29/2020,Minnesota,27,52987,1629,46.729553,-94.685900
2,07/28/2020,Minnesota,27,52321,1620,46.729553,-94.685900
3,07/27/2020,Minnesota,27,51843,1616,46.729553,-94.685900
4,07/26/2020,Minnesota,27,51193,1614,46.729553,-94.685900
...,...,...,...,...,...,...,...
8259,01/28/2020,Illinois,17,1,0,40.633125,-89.398528
8260,01/27/2020,Illinois,17,1,0,40.633125,-89.398528
8261,01/26/2020,Illinois,17,1,0,40.633125,-89.398528
8262,01/25/2020,Illinois,17,1,0,40.633125,-89.398528


In [23]:
state_final_latest=state_final_df.loc[state_final_df['date']==state_final_df['date'].max()]
state_final_latest.head()

,date,State,fips,cases,deaths,Latitude,Longitude
0,07/30/2020,Minnesota,27,53732,1634,46.729553,-94.685900
147,07/30/2020,Florida,12,461371,6585,27.664827,-81.515754
299,07/30/2020,Hawaii,15,1966,25,19.896766,-155.582782
446,07/30/2020,Guam,66,1323,6,13.444304,144.793731
584,07/30/2020,Alabama,1,85762,1565,32.318231,-86.902298


In [38]:
nyt_state_json_latest=state_final_latest.to_json(orient='records')
parsed=json.loads(nyt_state_json_latest)
print(nyt_state_json_latest)

[{"date":"07\/30\/2020","State":"Minnesota","fips":27,"cases":53732,"deaths":1634,"Latitude":46.729553,"Longitude":-94.6858998},{"date":"07\/30\/2020","State":"Florida","fips":12,"cases":461371,"deaths":6585,"Latitude":27.6648274,"Longitude":-81.5157535},{"date":"07\/30\/2020","State":"Hawaii","fips":15,"cases":1966,"deaths":25,"Latitude":19.8967662,"Longitude":-155.5827818},{"date":"07\/30\/2020","State":"Guam","fips":66,"cases":1323,"deaths":6,"Latitude":13.444304,"Longitude":144.793731},{"date":"07\/30\/2020","State":"Alabama","fips":1,"cases":85762,"deaths":1565,"Latitude":32.3182314,"Longitude":-86.902298},{"date":"07\/30\/2020","State":"Alaska","fips":2,"cases":3557,"deaths":21,"Latitude":64.2008413,"Longitude":-149.4936733},{"date":"07\/30\/2020","State":"Arizona","fips":4,"cases":170905,"deaths":3628,"Latitude":34.0489281,"Longitude":-111.0937311},{"date":"07\/30\/2020","State":"Arkansas","fips":5,"cases":41759,"deaths":442,"Latitude":35.20105,"Longitude":-91.8318334},{"date":"

In [39]:
with open('nyt_state_covid_latest.json','w') as f:
    json.dump(parsed, f, indent=4)

In [32]:
from flask_pymongo import PyMongo
from config import mongoURL 

from pymongo import MongoClient
import json
#client = MongoClient('localhost', 27017)
client = MongoClient(mongoURL)

In [40]:
nyt_db = client['nyt_covid_db']
collection_nyt_lt = nyt_db['NYT_state_covid_latest']
with open('nyt_state_covid_latest.json') as f:
    nyt_data1 = json.load(f)
collection_nyt_lt.insert_many(nyt_data1)

In [41]:
state_json=state_final_df.to_json(orient='records')
parsed=json.loads(state_json)

In [42]:
with open('nyt_state_covid_daily.json','w') as f:
    json.dump(parsed, f, indent=4)

In [43]:
nyt_db = client['nyt_covid_db']
collection_nyt = nyt_db['NYT_state_covid_daily']
with open('nyt_state_covid_daily.json') as f:
    nyt_data2 = json.load(f)
collection_nyt.insert_many(nyt_data2)

## Getting Lat & Long for County Data

In [112]:
county_df = county_df.rename(columns = {'county':'County'})
county_df = county_df[county_df.County != 'Unknown']
county_latest = county_df.loc[county_df['date'] == county_df['date'].max()]
nyc = county_latest.loc[county_latest['County'] == 'New York City'].index
#nyc
county_latest.at[384619, 'fips'] = 36061.0
#kansas city
county_latest.at[384308, 'fips'] = 38000.0
#joplin co
county_latest.at[384307, 'fips'] = 37592.0
county_latest = county_latest.astype({'fips':'int64'})
#county_latest_new = county_latest.loc[county_latest['fips'].isnull()]
county_latest

,date,County,state,fips,cases,deaths
385984,07/30/2020,Weston,Wyoming,56045,5,0
385972,07/30/2020,Laramie,Wyoming,56021,449,3
385960,07/30/2020,Winnebago,Wisconsin,55139,997,16
385981,07/30/2020,Teton,Wyoming,56039,331,1
385980,07/30/2020,Sweetwater,Wyoming,56037,237,2
...,...,...,...,...,...,...
382775,07/30/2020,Chambers,Alabama,1017,848,38
382774,07/30/2020,Calhoun,Alabama,1015,1336,8
382773,07/30/2020,Butler,Alabama,1013,747,36
382772,07/30/2020,Bullock,Alabama,1011,435,12


In [118]:
# county_list=county_latest['County'].tolist()
# county_list

In [119]:
#len(county_list)

In [120]:
# county_coords_df=pd.DataFrame(county_list,columns=['County'])

In [ ]:
#lat=[]
#long=[]

In [121]:
# county_only_df = pd.read_csv(lat_long_counties)
# county_only_df = county_only_df.rename(columns = {'Unnamed: 1': 'State'})

In [122]:
# county_only_df['GoogleSearch'] = county_only_df['county-state'].str.cat(county_only_df['State'], sep =" ") 
# county_only_df.head()

In [123]:
# county_series=county_only_df['GoogleSearch'].unique()
# county_list=county_series.tolist()
# county_list

In [124]:
# for i in county_list:
#     target_url=(f"https://maps.googleapis.com/maps/api/geocode/json?address={i}&key={geocode_key}")
#     geodata=requests.get(target_url).json()
#     #print(geodata)
#     try:
#         latitude=geodata['results'][0]['geometry']['location']['lat']
#         lat.append(latitude)
#         longitude=geodata['results'][0]['geometry']['location']['lng']
#         long.append( longitude)
#     except:
#         print(i)
#         lat.append('NaN')
#         long.append('NaN')

In [125]:
# county_series_final=county_only_df['county-state']
# state_series_final=county_only_df['State']

# county_list_final =county_series_final.tolist()
# state_list_final =state_series_final.tolist()
# county_info_dict = {"County": county_list_final, "state": state_list_final, "Latitude": lat, "Longitude": long}

In [126]:
# county_nyt_covid_info = pd.DataFrame(county_info_dict)
# county_nyt_covid_info.head()

In [ ]:
county_nyt_final_df=pd.merge(county_df,county_nyt_covid_info, on=['County', 'state'])

In [ ]:
county_nyt_final_df

In [ ]:
county_nyt_final_df = county_nyt_final_df.sort_values('date', ascending = False)
county_nyt_final_df.head()

In [ ]:
county_nyt_final_json = county_nyt_final_df.to_json(orient='records')
parsed =json.loads(county_nyt_final_json)

In [ ]:
with open('county_nyt_daily.json', 'w') as f:
    json.dump(parsed, f, indent=4)